In [1]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
import random
from datetime import date

In [2]:
pip install yahoo_fin


Note: you may need to restart the kernel to use updated packages.


In [3]:
from yahoo_fin.stock_info import get_data

In [63]:

# ***************************
# ***                     ***
# ***    SET VARIABLES    ***
# ***                     ***
# ***************************

days_in_future_that_result_is_calculated = 252
print(f"Days in future that result is calculated: {days_in_future_that_result_is_calculated}")

reversion_low_value = 12
print(f"reversion_low_value: {reversion_low_value}")

reversion_high_value = 20
print(f"reversion_high_value: {reversion_high_value}")

optimization_max_epochs = 5
print(f"optimization_max_epochs: {optimization_max_epochs}")

# Max 10:
optimization_max_hidden_layers = 5
print(f"optimization_max_hidden_layers: {optimization_max_hidden_layers}")

optimization_max_nodes = 5


activation_type = 'relu'

# Max 10:
num_layers = 2

num_epochs = 5

num_nodes = 5


# SCALE THE DATA? "Y" or "N":
scale_data = "Y"


# TICKER SYMBOLS TO PROCESS:

# 21 symbols to pair up, plus 2 result tickers to test output,
# so 210 unique pairs * 2 outputs = 420 combinations are 
# looped through and be tested

symbol_eval_list = [
                        '^GSPC',
                        '^DJI',
                        '^IXIC',
                        '^RUT',
                        'XLC',
                        'XLY',
                        'XLP',
                        'XLE',
                        'XLF',
                        'XLV',
                        'XLI',
                        'XLB',
                        'XLRE',
                        'XLK',
                        'XLU',
                        '^VIX',
                        '^VIX3M',
                        'CL=F',
                        '^TNX',
                        'BTC-USD',
                        '^CMC200',
                        ]

symbol_result_list = ['SPY'
                    , 'USO'
                     ]

input_symbol_01 = ""
input_symbol_02 = ""
input_symbol_03 = ""
output_count = 0

days_looking_back = 1

# Triple-loop through all of the symbol_eval_list and all of the symbol_result_list
# and determine all possible combinations. Write all combos to a list. 
# Then auto-run machine learning neural networks for each
# one of them by looping through the list that was created.
# Use the results to determine best candidates for further analysis.
# The result is 420 unique combinations to test.

# Outer loop through the smaller group of ticker symbols symbol_result_list
for x3 in symbol_result_list:
  input_symbol_03 = x3
  pairings = []
  for x1 in symbol_eval_list:
    # Then get a ticker symbol from symbol_eval_list
    input_symbol_01 = x1
    for x2 in symbol_eval_list:
      # get another symbol to pair up from ymbol_eval_list 
      input_symbol_02 = x2
      # if the two symbols are not the same, proceed
      if input_symbol_01 != input_symbol_02:
        pairing_found = 'N'
        # loop through the found pairings and check if that pairing already exists (also check the reverse order)
        for i in range(len(pairings)):
          if (pairing_found == 'N'):
            if (pairings[i] == [x1, x2]) | (pairings[i] == [x2, x1]):
              pairing_found = 'Y'
        # if no pairing has already been writted out, we can process it, so write it now so it won't be processed again and then do all the procesing
        if (pairing_found == 'N'):
          pairings.append([x1, x2])
          print(f"BEGIN PROCESS")

          # Load ticker1 historical data from Yahoo Finance API
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_01 = get_data(input_symbol_01, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_01_min_date = (df_01.iloc[0]["date"])

          # Load ticker2 historical data from Yahoo Finance API
          from datetime import date
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_02 = get_data(input_symbol_02, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_02_min_date = (df_02.iloc[0]["date"])

          # Load ticker3 historical data from Yahoo Finance API
          from datetime import date
          today = date.today()
          curr_date = today.strftime("%m/%d/%Y")
          df_03 = get_data(input_symbol_03, start_date="01/01/2000", end_date=curr_date, index_as_date = False, interval="1d")
          df_03_min_date = (df_03.iloc[0]["date"])

          # Find the first date in each dataframe and get the latest date, that way all dataframes start on the same date when concatinated
          latest_date = df_01_min_date
          if df_02_min_date > latest_date:
            latest_date = df_02_min_date
          if df_03_min_date > latest_date:
            latest_date = df_03_min_date
          print(f"latest_date: {latest_date}")
          df_01 = df_01[df_01.date >= latest_date]
          df_02 = df_02[df_02.date >= latest_date]
          df_03 = df_03[df_03.date >= latest_date]
          
          output_count+=1
          print(f"OUTPUT COUNT: {output_count}")
          # set the column names for this run, based on the ticker symbols
          col01 = (f"{input_symbol_01}_close")
          col02 = (f"{input_symbol_02}_close")
          col03 = (f"{input_symbol_03}_close")

          # clean and merge the 3 dataframes, and add some new cols
          df = df_01[['date', 'close']].copy()
          df = df.rename(columns={'close': col01})
          df = df.merge(df_02, on='date')
          df = df.rename(columns={'close': col02})
          df = df.drop(columns=['open', 'high', 'low', 'adjclose', 'volume', 'ticker'])
          df[input_symbol_01+'_percentage'] = ""
          df[input_symbol_02+'_percentage'] = ""
          df[input_symbol_01+'_percentage_boolean'] = ""
          df[input_symbol_02+'_percentage_boolean'] = ""
#           df['reversion_boolean'] = ""
          df = df.merge(df_03, on='date')
          df = df.rename(columns={'close': col03})
          df = df.drop(columns=['open', 'high', 'low', 'adjclose', 'volume', 'ticker'])
          df[input_symbol_03+'_future_result'] = ""
          df[input_symbol_03+'_future_result_boolean'] = ""

          # loop through the df to populate the new columns
          for i in range(len(df)):
            if (i < (len(df))):
                if (i > (days_looking_back)):
                    
                    df[input_symbol_01+'_percentage'][i] = ((df[col01][i]) - (df[col01][i-days_looking_back])) / (df[col01][i-days_looking_back])
                    if (df[input_symbol_01+'_percentage'][i] != "" ):
                        if (df[input_symbol_01+'_percentage'][i] > 0 ):
                            df[input_symbol_01+'_percentage_boolean'][i] = 1
                        else:
                            df[input_symbol_01+'_percentage_boolean'][i] = 0
                            
                    df[input_symbol_02+'_percentage'][i] = ((df[col02][i]) - (df[col02][i-days_looking_back])) / (df[col02][i-days_looking_back])
                    if (df[input_symbol_02+'_percentage'][i] != "" ):
                        if (df[input_symbol_02+'_percentage'][i] > 0 ):
                            df[input_symbol_02+'_percentage_boolean'][i] = 1
                        else:
                            df[input_symbol_02+'_percentage_boolean'][i] = 0
            if (i < (len(df)-days_in_future_that_result_is_calculated)):
              df[input_symbol_03+'_future_result'][i] = ((df[col03][i+days_in_future_that_result_is_calculated]-df[col03][i])/df[col03][i])
 

                       
                        

          df = df[df[input_symbol_03+'_future_result'] != ""]

            
#           df = df[df[input_symbol_01+'_percentage'] != ""]
#           df = df[df[input_symbol_02+'_percentage'] != ""]
        
        

          # For the model, pick which cols will be fed into 'X' as inputs
          input_column_list_for_X = [
              input_symbol_01+"_close"
            , 
            input_symbol_02+"_close"
            # , 
            # "percentage"
            # ,
            # "percentage_boolean"
            # ,
            # "reversion_boolean"
            ]
          print (f"input_column_list_for_X: {input_column_list_for_X}")

          # For the model, pick which col will be the 'y' result
          output_col_for_y = [
              # "future_result"
              input_symbol_03+"_future_result_boolean"
              ]
          print (f"output_col_for_y: {output_col_for_y}")

        # make sure numeric
          df[input_symbol_03+"_future_result"] = pd.to_numeric(df[input_symbol_03+"_future_result"])

        # set more new cols
          for i in range(len(df)):
#               if (df[input_symbol_03+'_future_result'][i] != "" ):
                  if (df[input_symbol_03+'_future_result'][i] > 0):
                    df[input_symbol_03+'_future_result_boolean'][i] = 1
                  else:
                    df[input_symbol_03+'_future_result_boolean'][i] = 0

          df[input_symbol_03+"_future_result_boolean"] = pd.to_numeric(df[input_symbol_03+"_future_result_boolean"])

          df = df[df[input_symbol_01+'_percentage'] != ""]
          df = df[df[input_symbol_02+'_percentage'] != ""]
        
        # make sure numeric
          df[input_symbol_01+'_percentage'] = pd.to_numeric(df[input_symbol_01+'_percentage'])
          df[input_symbol_01+'_percentage_boolean'] = pd.to_numeric(df[input_symbol_01+'_percentage_boolean'])
          df[input_symbol_02+'_percentage'] = pd.to_numeric(df[input_symbol_02+'_percentage'])
          df[input_symbol_02+'_percentage_boolean'] = pd.to_numeric(df[input_symbol_02+'_percentage_boolean'])

            
            
          # Remove target 'y' from features data
          y = df[output_col_for_y].values

          # Pick columns to use for input 'X'
          X = df[input_column_list_for_X].values

          # Split the preprocessed data into a training and testing dataset
          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

          # Create a StandardScaler instances
          scaler = StandardScaler()

          # Fit the StandardScaler
          X_scaler = scaler.fit(X_train)

          # Scale the data
          if scale_data == "N":
            import numpy as np
            X_train = np.asarray(X_train).astype(np.float64)
            X_train_scaled = X_train
            X_test = np.asarray(X_test).astype(np.float64)
            X_test_scaled = X_test
          else:
            X_train_scaled = X_scaler.transform(X_train)
            X_test_scaled = X_scaler.transform(X_test)

          # Compile, Train and Evaluate the Model

          number_input_features = len(X_train[0])

          nn = tf.keras.models.Sequential()

          # First hidden layer
          hidden_nodes_layer1 = num_nodes
          nn.add(
              tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_type)
          )
          if num_layers >= 2:
            # second hidden layer
            hidden_nodes_layer2 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation_type))
          if num_layers >= 3:
            # hidden layer
            hidden_nodes_layer3 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation_type))
          if num_layers >= 4:
            # hidden layer
            hidden_nodes_layer4 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation_type))
          if num_layers >= 5:
            # hidden layer
            hidden_nodes_layer5 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation=activation_type))
          if num_layers >= 6:
            # hidden layer
            hidden_nodes_layer6 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation=activation_type))
          if num_layers >= 7:
            # hidden layer
            hidden_nodes_layer7 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation=activation_type))
          if num_layers >= 8:
            # hidden layer
            hidden_nodes_layer8 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer8, activation=activation_type))
          if num_layers >= 9:
            # hidden layer
            hidden_nodes_layer9 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer9, activation=activation_type))
          if num_layers >= 10:
            # hidden layer
            hidden_nodes_layer10 = num_nodes
            nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer10, activation=activation_type))

          # Output layer
          nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

          # Check the structure of the model
          nn.summary()

          # Compile the model
          nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
          # Train the model
          fit_model = nn.fit(X_train_scaled,y_train,epochs=num_epochs)

          # Evaluate the model using the test data
          model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

          # Need to document well the RESULTS from EACH run - this output will be just one of hundreds of loops in the output
          print(f"RESULTS")
          print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
          print(f"Symbol 01: {input_symbol_01}")
          print(f"Symbol 02: {input_symbol_02}")
          print(f"Symbol 03: {input_symbol_03}")
          print(f"Scale Data = {scale_data}")
          print(f"activation_type = {activation_type}")
          print(f"num_epochs = {num_epochs}")
          print(f"num_layers = {num_layers}")
          print(f"Layer 1 Nodes = {hidden_nodes_layer1}")
          if num_layers >= 2:
            print(f"Layer 2 Nodes = {hidden_nodes_layer2}")
          if num_layers >= 3:
            print(f"Layer 3 Nodes: {hidden_nodes_layer3}")
          if num_layers >= 4:
            print(f"Layer 4 Nodes = {hidden_nodes_layer4}")
          if num_layers >= 5:
            print(f"Layer 5 Nodes = {hidden_nodes_layer5}")
          if num_layers >= 6:
            print(f"Layer 6 Nodes = {hidden_nodes_layer6}")
          if num_layers >= 7:
            print(f"Layer 7 Nodes = {hidden_nodes_layer7}")
          if num_layers >= 8:
            print(f"Layer 8 Nodes = {hidden_nodes_layer8}")
          if num_layers >= 9:
            print(f"Layer 9 Nodes = {hidden_nodes_layer9}")
          if num_layers >= 10:
            print(f"Layer 10 Nodes = {hidden_nodes_layer10}")
          
          print("")
          print("")
          print("")

Days in future that result is calculated: 252
reversion_low_value: 12
reversion_high_value: 20
optimization_max_epochs: 5
optimization_max_hidden_layers: 5
BEGIN PROCESS
latest_date: 2000-01-03 00:00:00
OUTPUT COUNT: 1


C:\Users\mlech\AppData\Local\Temp\ipykernel_20876\4089011593.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[input_symbol_03+'_future_result'][i] = ((df[col03][i+days_in_future_that_result_is_calculated]-df[col03][i])/df[col03][i])
C:\Users\mlech\AppData\Local\Temp\ipykernel_20876\4089011593.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[input_symbol_01+'_percentage'][i] = ((df[col01][i]) - (df[col01][i-days_looking_back])) / (df[col01][i-days_looking_back])
C:\Users\mlech\AppData\Local\Temp\ipykernel_20876\4089011593.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

input_column_list_for_X: ['^GSPC_close', '^DJI_close']
output_col_for_y: ['SPY_future_result_boolean']


C:\Users\mlech\AppData\Local\Temp\ipykernel_20876\4089011593.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[input_symbol_03+'_future_result_boolean'][i] = 0
C:\Users\mlech\AppData\Local\Temp\ipykernel_20876\4089011593.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[input_symbol_03+'_future_result_boolean'][i] = 1


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 5)                 15        
                                                                 
 dense_32 (Dense)            (None, 5)                 30        
                                                                 
 dense_33 (Dense)            (None, 1)                 6         
                                                                 
Total params: 51 (204.00 Byte)
Trainable params: 51 (204.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5


KeyboardInterrupt: 

In [64]:
df.head(5724)

date  ^GSPC_close    ^DJI_close  ^GSPC_percentage  ^DJI_percentage  \
2    2000-01-05  1402.109985  11122.650391          0.001922         0.011340   
3    2000-01-06  1403.449951  11253.259766          0.000956         0.011743   
4    2000-01-07  1441.469971  11522.559570          0.027090         0.023931   
5    2000-01-10  1457.599976  11572.200195          0.011190         0.004308   
6    2000-01-11  1438.560059  11511.080078         -0.013063        -0.005282   
...         ...          ...           ...               ...              ...   
5717 2022-09-22  3757.989990  30076.679688         -0.008428        -0.003548   
5718 2022-09-23  3693.229980  29590.410156         -0.017233        -0.016168   
5719 2022-09-26  3655.040039  29260.810547         -0.010341        -0.011139   
5720 2022-09-27  3647.290039  29134.990234         -0.002120        -0.004300   
5721 2022-09-28  3719.040039  29683.740234          0.019672         0.018835   

      ^GSPC_percentage_boolean  ^DJI_percentage_boolean   SPY_close  \
2                            1                        1  140.000000   
3                            1                        1  137.750000   
4                            1                        1  145.750000   
5                            1                        1  146.250000   
6                            0                        0  144.500000   
...                        ...                      ...         ...   
5717                         0                        0  374.220001   
5718                         0                        0  367.950012   
5719                         0                        0  364.309998   
5720                         0                        0  363.380005   
5721                         1                        1  370.529999   

      SPY_future_result  SPY_future_result_boolean  
2             -0.046094                          0  
3             -0.062160                          0  
4             -0.106775                          0  
5             -0.112179                          0  
6             -0.085640                          0  
...                 ...                        ...  
5717           0.155016                          1  
5718           0.157440                          1  
5719           0.169471                          1  
5720           0.179261                          1  
5721           0.153699                          1  

[5720 rows x 10 columns]